In [8]:
# Basic module
import numpy as np
import pandas as pd

# Graphic module
import matplotlib.pyplot as plt
import seaborn as sns

# Chi-Square module
from scipy.stats import chi2, chi2_contingency, pointbiserialr  # Not exist in basic anaconda. Install need(pip install scipy or conda install scipy)
from scipy.stats.contingency import association

## Summary

범주형 데이터 그룹화 전, 상관관계가 있는 컬럼 확인용

## Data loading

### Load

In [2]:
labeled_data = pd.read_csv("./result_data/labeling_data.csv", header=0, index_col=0, encoding='utf-8')

### Pre-processing

In [3]:
# Joined 상태 Stayed로 변경
labeled_data_cp = labeled_data.copy()

labeled_data_cp['Customer Status'][labeled_data_cp['Customer Status'] == 'Joined'] = 'Stayed'

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_17100\2752292322.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_data_cp['Customer Status'][labeled_data_cp['Customer Status'] == 'Joined'] = 'Stayed'


## Correlation

In [4]:
# object형, 범주형 데이터 추출
categorical_column_list = []
for i in range(len(labeled_data_cp.columns)):
    if labeled_data_cp[labeled_data_cp.columns[i]].dtype == 'object':
        if len(labeled_data_cp[labeled_data_cp.columns[i]].unique()) <= 100 and len(labeled_data_cp[labeled_data_cp.columns[i]].unique()) != 1:
        # if len(labeled_data_cp[labeled_data_cp.columns[i]].unique()) != len(labeled_data_cp):
            categorical_column_list.append(labeled_data_cp.columns[i])

In [9]:
# Chi-Sqaure, P-value , Cramer's V 계산
chi_SQ_df = pd.DataFrame(columns=categorical_column_list, index=categorical_column_list)
chi_Pv_df = pd.DataFrame(columns=categorical_column_list, index=categorical_column_list)
chi_CV_df = pd.DataFrame(columns=categorical_column_list, index=categorical_column_list)
# chi, p가 하나의 값으로 나옴

for col_name in categorical_column_list:
    for col_name_2 in categorical_column_list:
        if col_name == col_name_2:
            # 동일한 값 끼린 계산하지 않기 위하여 0으로 처리
            chi_SQ_df[col_name][col_name_2] = 0
            chi_Pv_df[col_name][col_name_2] = 0
            chi_CV_df[col_name][col_name_2] = 0
            continue

        ct_result = pd.crosstab(labeled_data_cp[str(col_name)], labeled_data_cp[str(col_name_2)])

        chi, p, dof, expected = chi2_contingency(ct_result)  # 카이-스퀘어 계산
        rows = [row.to_list() for i, row in ct_result.iterrows()]
        corr = association(rows, method="cramer")
        
        chi_SQ_df[col_name][col_name_2] = chi
        chi_Pv_df[col_name][col_name_2] = p
        chi_CV_df[col_name][col_name_2] = corr


d:\ProgramData\anaconda3\lib\site-packages\scipy\stats\contingency.py:410: RuntimeWarning: invalid value encountered in double_scalars
  value = phi2 / min(n_cols - 1, n_rows - 1)
d:\ProgramData\anaconda3\lib\site-packages\scipy\stats\contingency.py:410: RuntimeWarning: invalid value encountered in double_scalars
  value = phi2 / min(n_cols - 1, n_rows - 1)
d:\ProgramData\anaconda3\lib\site-packages\scipy\stats\contingency.py:410: RuntimeWarning: invalid value encountered in double_scalars
  value = phi2 / min(n_cols - 1, n_rows - 1)
d:\ProgramData\anaconda3\lib\site-packages\scipy\stats\contingency.py:410: RuntimeWarning: invalid value encountered in double_scalars
  value = phi2 / min(n_cols - 1, n_rows - 1)
d:\ProgramData\anaconda3\lib\site-packages\scipy\stats\contingency.py:410: RuntimeWarning: invalid value encountered in double_scalars
  value = phi2 / min(n_cols - 1, n_rows - 1)
d:\ProgramData\anaconda3\lib\site-packages\scipy\stats\contingency.py:410: RuntimeWarning: invalid v

In [7]:
# 상관관계 표기를 위한 함수 작성
def p_value_H_checker(p_value):
    if p_value == 0:
        R_h0 = '동일'
    elif p_value < 0.05:
        R_h0 = '상관'
    else:
        R_h0 = '독립'

    return R_h0

In [8]:
# 함수 적용하여 표기 변경
chi_Pv_H_df = chi_Pv_df.applymap(p_value_H_checker)

In [9]:
chi_SQ_df

,Gender,Under 30,Senior Citizen,Married,Dependents,Referred a Friend,Offer,Phone Service,Multiple Lines,Internet Service,...,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Customer Status,Churn Label,Churn Category,Churn Reason
Gender,0,0.047848,0.015604,0.016356,0.16858,0.106614,7.864231,0.254185,0.465078,0.227311,...,0.732173,0.378214,0.000838,0.321745,0.925736,1.316404,0.484083,0.484083,2.383342,12.734062
Under 30,0.047848,0,336.968531,1.539443,11.97197,1.829888,4.131631,0.476089,8.591482,8.802009,...,0.629525,106.931053,6.940505,1.703887,10.442064,14.710973,20.690894,20.690894,2.080688,14.763698
Senior Citizen,0.015604,336.968531,0,1.824079,213.084288,0.612756,34.712805,0.442352,143.133693,233.995814,...,101.049313,153.063275,137.585298,7.992704,171.700988,158.609282,159.4263,159.4263,3.245505,41.840552
Married,0.016356,1.539443,1.824079,0,928.525119,6352.311469,465.379602,2.089728,141.554196,0.000515,...,96.60999,55.040878,2.407678,556.233818,1.498722,28.745395,158.733382,158.733382,4.094595,21.005881
Dependents,0.16858,11.97197,213.084288,928.525119,0,844.815703,62.576262,0.000021,5.198774,207.636196,...,37.55121,9.189336,114.736824,214.714736,99.10114,73.824604,433.734379,433.734379,9.093792,22.195012
Referred a Friend,0.106614,1.829888,0.612756,6352.311469,844.815703,0,431.867001,1.57119,123.373163,0.043869,...,93.200564,56.565953,2.465388,526.381812,0.822219,25.97443,155.940342,155.940342,8.009297,25.738373
Offer,7.864231,4.131631,34.712805,465.379602,62.576262,431.867001,0,5.703308,402.167744,19.888029,...,262.531251,186.803099,12.143275,1629.682099,3.165015,75.624115,481.725489,481.725489,32.760657,115.314511
Phone Service,0.254185,0.476089,0.442352,2.089728,0.000021,1.57119,5.703308,0,549.035796,207.455145,...,7.42372,10.575391,104.627437,0.05196,1.806686,5.92823,0.915033,0.915033,4.712565,25.556152
Multiple Lines,0.465078,8.591482,143.133693,141.554196,5.198774,123.373163,402.167744,549.035796,0,311.231393,...,470.470645,262.512276,178.866528,105.454589,187.671516,164.29692,11.143251,11.143251,7.748812,39.707321
Internet Service,0.227311,8.802009,233.995814,0.000515,207.636196,0.043869,19.888029,207.455145,311.231393,0,...,1232.474436,1062.099553,4018.582615,288.711484,724.192094,525.284576,364.519799,364.519799,108.473793,296.150058


In [10]:
chi_Pv_df

,Gender,Under 30,Senior Citizen,Married,Dependents,Referred a Friend,Offer,Phone Service,Multiple Lines,Internet Service,...,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Customer Status,Churn Label,Churn Category,Churn Reason
Gender,0,0.826852,0.900589,0.898234,0.681377,0.744032,0.163879,0.614143,0.49526,0.633525,...,0.39218,0.538561,0.976903,0.851401,0.335973,0.517781,0.486579,0.486579,0.66564,0.851902
Under 30,0.826852,0,0.0,0.214701,0.00054,0.176141,0.530624,0.490199,0.003377,0.003009,...,0.42753,0.0,0.008427,0.426585,0.001232,0.000639,0.000005,0.000005,0.720921,0.737493
Senior Citizen,0.900589,0.0,0,0.176829,0.0,0.433752,0.000002,0.505989,0.0,0.0,...,0.0,0.0,0.0,0.018383,0.0,0.0,0.0,0.0,0.517613,0.001862
Married,0.898234,0.214701,0.176829,0,0.0,0.0,0.0,0.148292,0.0,0.981901,...,0.0,0.0,0.120741,0.0,0.220868,0.000001,0.0,0.0,0.393355,0.336476
Dependents,0.681377,0.00054,0.0,0.0,0,0.0,0.0,0.996302,0.022603,0.0,...,0.0,0.002434,0.0,0.0,0.0,0.0,0.0,0.0,0.058797,0.274659
Referred a Friend,0.744032,0.176141,0.433752,0.0,0.0,0,0.0,0.210034,0.0,0.834098,...,0.0,0.0,0.116379,0.0,0.364532,0.000002,0.0,0.0,0.091238,0.137646
Offer,0.163879,0.530624,0.000002,0.0,0.0,0.0,0,0.336167,0.0,0.001312,...,0.0,0.0,0.032876,0.0,0.674563,0.0,0.0,0.0,0.035838,0.076685
Phone Service,0.614143,0.490199,0.505989,0.148292,0.996302,0.210034,0.336167,0,0.0,0.0,...,0.006437,0.001146,0.0,0.974355,0.178906,0.051606,0.338783,0.338783,0.318081,0.143038
Multiple Lines,0.49526,0.003377,0.0,0.0,0.022603,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000843,0.000843,0.101225,0.003574
Internet Service,0.633525,0.003009,0.0,0.981901,0.0,0.834098,0.001312,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
chi_Pv_H_df

,Gender,Under 30,Senior Citizen,Married,Dependents,Referred a Friend,Offer,Phone Service,Multiple Lines,Internet Service,...,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Customer Status,Churn Label,Churn Category,Churn Reason
Gender,동일,독립,독립,독립,독립,독립,독립,독립,독립,독립,...,독립,독립,독립,독립,독립,독립,독립,독립,독립,독립
Under 30,독립,동일,상관,독립,상관,독립,독립,독립,상관,상관,...,독립,상관,상관,독립,상관,상관,상관,상관,독립,독립
Senior Citizen,독립,상관,동일,독립,상관,독립,상관,독립,상관,상관,...,상관,상관,상관,상관,상관,상관,상관,상관,독립,상관
Married,독립,독립,독립,동일,상관,동일,상관,독립,상관,독립,...,상관,상관,독립,상관,독립,상관,상관,상관,독립,독립
Dependents,독립,상관,상관,상관,동일,상관,상관,독립,상관,상관,...,상관,상관,상관,상관,상관,상관,상관,상관,독립,독립
Referred a Friend,독립,독립,독립,동일,상관,동일,상관,독립,상관,독립,...,상관,상관,독립,상관,독립,상관,상관,상관,독립,독립
Offer,독립,독립,상관,상관,상관,상관,동일,독립,상관,상관,...,상관,상관,상관,동일,독립,상관,상관,상관,상관,독립
Phone Service,독립,독립,독립,독립,독립,독립,독립,동일,상관,상관,...,상관,상관,상관,독립,독립,독립,독립,독립,독립,독립
Multiple Lines,독립,상관,상관,상관,상관,상관,상관,상관,동일,상관,...,상관,상관,상관,상관,상관,상관,상관,상관,독립,상관
Internet Service,독립,상관,상관,독립,상관,독립,상관,상관,상관,동일,...,상관,상관,동일,상관,상관,상관,상관,상관,상관,상관


In [10]:
chi_CV_df

,Gender,Under 30,Senior Citizen,Married,Dependents,Referred a Friend,Offer,Phone Service,Multiple Lines,Internet Service,...,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Customer Status,Churn Label,Churn Category,Churn Reason
Gender,0,0.002962,0.001874,0.001808,0.005229,0.004176,0.033416,0.006488,0.008414,0.006026,...,0.010487,0.007625,0.000648,0.006759,0.011754,0.013671,0.008612,0.008612,0.03571,0.082543
Under 30,0.002962,0,0.219216,0.01514,0.041651,0.016476,0.02422,0.008823,0.035287,0.035784,...,0.009819,0.12359,0.031771,0.015554,0.038867,0.045703,0.054604,0.054604,0.033366,0.088878
Senior Citizen,0.001874,0.219216,0,0.016479,0.174396,0.009714,0.070205,0.008576,0.142948,0.182742,...,0.120176,0.147823,0.140179,0.033687,0.15653,0.150067,0.150889,0.150889,0.041671,0.149622
Married,0.001808,0.01514,0.016479,0,0.36343,0.949987,0.257054,0.017706,0.142057,0.000615,...,0.117412,0.0887,0.018792,0.281028,0.014877,0.063886,0.150448,0.150448,0.046806,0.106015
Dependents,0.005229,0.041651,0.174396,0.36343,0,0.346678,0.09426,0.000625,0.02751,0.17211,...,0.073364,0.036474,0.127995,0.174603,0.118963,0.102382,0.248542,0.248542,0.069754,0.108974
Referred a Friend,0.004176,0.016476,0.009714,0.949987,0.346678,0,0.247626,0.015418,0.132641,0.002842,...,0.115328,0.089917,0.019014,0.273383,0.011095,0.060729,0.149122,0.149122,0.065462,0.117351
Offer,0.033416,0.02422,0.070205,0.257054,0.09426,0.247626,0,0.028457,0.23896,0.053139,...,0.193069,0.16286,0.041523,0.34014,0.021199,0.073272,0.26153,0.26153,0.066198,0.111084
Phone Service,0.006488,0.008823,0.008576,0.017706,0.000625,0.015418,0.028457,0,0.27969,0.172209,...,0.032959,0.039252,0.122395,0.002716,0.016505,0.029012,0.011942,0.011942,0.050214,0.116935
Multiple Lines,0.008414,0.035287,0.142948,0.142057,0.02751,0.132641,0.23896,0.27969,0,0.210564,...,0.258751,0.193362,0.159669,0.122364,0.16353,0.152734,0.040102,0.040102,0.064389,0.145757
Internet Service,0.006026,0.035784,0.182742,0.000615,0.17211,0.002842,0.053139,0.172209,0.210564,0,...,0.418675,0.388693,0.755734,0.202467,0.321013,0.273098,0.22789,0.22789,0.240912,0.398062


In [12]:
# 독립/상관 개수 파악
# 다른 컬럼에 어느 정도의 영향을 끼치는지 확인용
Pv_H_result_df = pd.DataFrame(columns=list(chi_Pv_H_df.columns), index=['독립', '동일', '상관'])

for col_name in chi_Pv_H_df.columns:
    concating_df = chi_Pv_H_df[col_name].value_counts().sort_index().to_frame()
    
    Pv_H_result_df[col_name] = concating_df

Pv_H_result_df.fillna(0)

,Gender,Under 30,Senior Citizen,Married,Dependents,Referred a Friend,Offer,Phone Service,Multiple Lines,Internet Service,...,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Customer Status,Churn Label,Churn Category,Churn Reason
독립,25.0,13,5,10,6,10,5,15,2,3,...,4,3,3,4,7,2,4,4,17,14
동일,1.0,1,1,2,1,2,2,1,1,3,...,3,3,3,4,1,1,3,3,2,2
상관,0.0,12,20,14,19,14,19,10,23,20,...,19,20,20,18,18,23,19,19,7,10


In [13]:
Pv_H_result_df.fillna(0).T.sort_values(by=['상관', '독립'], ascending=False)

,독립,동일,상관
Multiple Lines,2.0,1.0,23.0
Payment Method,2.0,1.0,23.0
Online Security,3.0,1.0,22.0
Online Backup,4.0,1.0,21.0
Device Protection Plan,4.0,1.0,21.0
Premium Tech Support,4.0,1.0,21.0
Senior Citizen,5.0,1.0,20.0
Internet Service,3.0,3.0,20.0
Internet Type,3.0,3.0,20.0
Streaming Music,3.0,3.0,20.0


In [12]:
grouping_list = ['Customer Status',
'Contract',
'Churn Label',
'Internet Type',
'Offer',
'Dependents',
'Internet Service',
'Payment Method',
'Paperless Billing',
'Online Security',
'Unlimited Data',
'Premium Tech Support',
'Senior Citizen',
'Married',
'Referred a Friend',
'Online Backup',
'Device Protection Plan',
'Streaming TV',
'Streaming Movies',
'Under 30',
'Streaming Music',
'Multiple Lines',
]

In [19]:
# 최대 Chi-SQ 값 확인
for i in grouping_list:
    print(f"{i} : {chi_SQ_df[i].loc[chi_SQ_df[i] == chi_SQ_df[i].max()].index[0]}")

Customer Status : Churn Label
Contract : Offer
Churn Label : Customer Status
Internet Type : Internet Service
Offer : Contract
Dependents : Married
Internet Service : Internet Type
Payment Method : Internet Type
Paperless Billing : Internet Type
Online Security : Internet Type
Unlimited Data : Internet Type
Premium Tech Support : Internet Type
Senior Citizen : Internet Type
Married : Referred a Friend
Referred a Friend : Married
Online Backup : Internet Type
Device Protection Plan : Streaming Movies
Streaming TV : Streaming Movies
Streaming Movies : Streaming Music
Under 30 : Senior Citizen
Streaming Music : Streaming Movies
Multiple Lines : Internet Type


In [24]:
# 최소 p-value 확인
for i in grouping_list:
    # print(f"{i} : {chi_Pv_df[i].loc[chi_Pv_df[i] == chi_Pv_df[i].min()].index[0]}")
    if chi_Pv_df[i].sort_values()[0] == 0:
        column_name = chi_Pv_df[i].sort_values()[1]
        print(f"{i} : {chi_Pv_df[i].loc[chi_Pv_df[i] == column_name].index[0]}")
    else:
        print(f"{i} : {chi_Pv_df[i].loc[chi_Pv_df[i] == chi_Pv_df[i].min()].index[0]}")

Customer Status : Contract
Contract : Offer
Churn Label : Contract
Internet Type : Internet Service
Offer : Offer
Dependents : Married
Internet Service : Internet Service
Payment Method : Internet Type
Paperless Billing : Internet Type
Online Security : Internet Type
Unlimited Data : Internet Service
Premium Tech Support : Internet Type
Senior Citizen : Internet Type
Married : Married
Referred a Friend : Married
Online Backup : Internet Service
Device Protection Plan : Streaming Movies
Streaming TV : Streaming TV
Streaming Movies : Streaming TV
Under 30 : Senior Citizen
Streaming Music : Streaming TV
Multiple Lines : Internet Type


In [14]:
# Cramer's V 값(범주형 상관관계) 최대치 확인
for i in grouping_list:
    print(f"{i} : {chi_CV_df[i].loc[chi_CV_df[i] == chi_CV_df[i].max()].index[0]}")

Customer Status : Churn Label
Contract : Customer Status
Churn Label : Customer Status
Internet Type : Internet Service
Offer : Contract
Dependents : Married
Internet Service : Internet Type
Payment Method : Internet Service
Paperless Billing : Internet Type
Online Security : Internet Type
Unlimited Data : Internet Type
Premium Tech Support : Internet Type
Senior Citizen : Internet Type
Married : Referred a Friend
Referred a Friend : Married
Online Backup : Internet Type
Device Protection Plan : Streaming Movies
Streaming TV : Streaming Movies
Streaming Movies : Streaming Music
Under 30 : Senior Citizen
Streaming Music : Streaming Movies
Multiple Lines : Internet Type


## Testing

In [14]:
labeled_data_cp['Age'][labeled_data_cp['Senior Citizen']=='Yes'].min()

65

In [15]:
chi_SQ_df['Gender'].loc[chi_SQ_df['Gender'] == chi_SQ_df['Gender'].max()].index[0]

'Churn Reason'